In [ ]:
from pathlib import Path
from tqdm import tqdm
from facenet_pytorch import InceptionResnetV1, MTCNN
from PIL import Image
import numpy  as np
import torch
import math
import warnings
from multiprocessing import Pool
warnings.filterwarnings('ignore')

## Image cropping and aligning
Load extracted 224x224 images from disk and process batch-wise

In [3]:
images_dir = Path("/mnt/U/Datasets/lrs3pretrain/intermediate/train_images")
outpath = images_dir.parent / "train_cropped_images"

In [4]:
speakers = list(images_dir.glob("*"))
im_per_spk = {x.name: list(x.glob("*.jpg")) for x in speakers}
im_paths = []
for speaker, ims in im_per_spk.items():
    for im in ims:
        out = outpath / speaker / im.name
        if not out.exists():
            im_paths += [(im, str(out))]

In [5]:
mtcnn = MTCNN(margin=10, post_process=False, device="cuda")

Use `multiprocessing.Pool` to deal with IO bottleneck in image loading

In [6]:
batch_size = 512
p = Pool(16)
skipped = im_paths.pop(0)
pbar = tqdm(total=len(im_paths))
for i in range(math.ceil(len(im_paths)/batch_size)):
    ims, paths = list(zip(*im_paths[i*batch_size:(i+1)*batch_size]))
    images = list(p.map(Image.open, ims))
    mtcnn(images, save_path=paths)
    pbar.update(batch_size)


734720it [2:35:30, 95.05it/s]                              